In [1]:
import MySQLdb
import pandas as pd
import json

In [2]:
def parse_timezone(property_string):
    property_dict = json.loads(property_string)
    try:
        timezone_string = property_dict['raw_timezone']
    except KeyError: #deafult timezone is Moscow +3UTC (if no tz mentioned, so assume tz="+3UTC")
        timezone_string = '+3'

    sign = 2 *(timezone_string[0]=='+') - 1 # 1 -- eastern hemisphere, -1 -- western hemisphere
    return sign * int(timezone_string[1:])

## Loading station_def table from lmeteo3

In [3]:
db = MySQLdb.connect("192.168.1.230", 
                     user='lm_ro', 
                     passwd='lm_rovtntj',
                     port=3306)

# utf-8 encoding
db.set_character_set("utf8")
dbc = db.cursor()

sql_query = "select objects.id as 'station_id', roads.id as 'road_id', places.address, " + \
            "concat('км ', places.address, '+',places.meters, " + \
            " ' а/д ' , roads.short_name) as 'name' ," +\
            "places.latitude, places.longitude, places.height as 'altitude', " + \
            "objects.properties " + \
            "from cup_system3.objects, cup_system3.places, cup_system3.roads, lmeteo3.station_def " + \
            "where lmeteo3.station_def.id = objects.id " + \
            "and objects.place_id = places.id " + \
            "and places.road_id = roads.id;"

# SELECT for cyrillic fields
dbc.execute(sql_query)

# getting the results
data = dbc.fetchall()

columns = ['station_id', 'road_id', 'km', 'name', 'longitude', 'latitude', 'altitude', 'properties']

# creating pandas table with column names from above
df = pd.DataFrame(list(data), columns=columns)

#df.to_csv('/home/ndsviriden/data_csv/station_def.csv')
df.head()

,station_id,road_id,km,name,longitude,latitude,altitude,properties
0,103,7,80,"км 80+000 а/д М-4 ""Дон""",55.278889,38.492500,0,"{""place"":"""",""direction"":""0"",""port"":""443"",""logi..."
1,104,54,47,"км 47+000 а/д М-5 ""Урал""",55.491667,38.157222,0,"{""place"":"""",""direction"":""0"",""port"":""443"",""logi..."
2,105,54,28,"км 28+000 а/д М-5 ""Урал""",55.605556,37.978889,0,"{""place"":"""",""direction"":""0"",""proto_type"":""F4"",..."
3,106,54,1369,"км 1369+260 а/д М-5 ""Урал""",54.644467,54.545000,188,"{""place"":"""",""direction"":""0"",""port"":""443"",""logi..."
4,107,54,1401,"км 1401+900 а/д М-5 ""Урал""",54.730047,55.011639,129,"{""place"":"""",""direction"":""0"",""port"":""443"",""logi..."


In [4]:
df['timezone'] = df.properties.apply(parse_timezone)
del df['properties']
df.head()

,station_id,road_id,km,name,longitude,latitude,altitude,timezone
0,103,7,80,"км 80+000 а/д М-4 ""Дон""",55.278889,38.492500,0,3
1,104,54,47,"км 47+000 а/д М-5 ""Урал""",55.491667,38.157222,0,3
2,105,54,28,"км 28+000 а/д М-5 ""Урал""",55.605556,37.978889,0,3
3,106,54,1369,"км 1369+260 а/д М-5 ""Урал""",54.644467,54.545000,188,3
4,107,54,1401,"км 1401+900 а/д М-5 ""Урал""",54.730047,55.011639,129,3


## Writing the result table into file

In [7]:
df.to_csv('/mnt/HARD/MinMax94/data/data_all/CSV/stations_mm94_def.csv', index=False)